---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    unis = open("university_towns.txt", "r").read()
    unis = unis.split("\n")
    data = []
    curr_state = str()
    for line in unis[:-1]:
        if "[edit]" in line:
            curr_state = str()
            curr_state = str(line[:-6])
            continue
        elif " (" in line:
            data.append([curr_state, line.split(" (")[0]])
        else:
            data.append([curr_state, line])
    df = pd.DataFrame(data, columns=["State","RegionName"])
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    data = pd.read_excel("gdplev.xls", header=None, skiprows=220, usecols=[4,6])
    data.columns = ["Quarter", "GDP"]
    pre2_row = next(data.itertuples())
    pre1_row = next(data.itertuples())
    for curr_row in data.itertuples():
        if (pre2_row.GDP > pre1_row.GDP) and (pre1_row.GDP > curr_row.GDP):
            decline = pre1_row.Quarter
            return decline
        else:
            pre2_row = pre1_row
            pre1_row = curr_row
    
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    data = pd.read_excel("gdplev.xls", header=None, skiprows=220, usecols=[4,6])
    data.columns = ["Quarter", "GDP"]
    data = data[data["Quarter"] > get_recession_start()]
    pre2_row = next(data.itertuples())
    pre1_row = next(data.itertuples())
    for curr_row in data.itertuples():
        if (pre2_row.GDP < pre1_row.GDP) and (pre1_row.GDP < curr_row.GDP):
            growth = curr_row.Quarter
            return growth
        else:
            pre2_row = pre1_row
            pre1_row = curr_row
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    data = pd.read_excel("gdplev.xls", header=None, skiprows=220, usecols=[4,6])
    data.columns = ["Quater", "GDP"]
    data = data[data["Quater"] >= get_recession_start()]
    data = data[data["Quater"] <= get_recession_end()]
    res = data[data["GDP"] == min(data["GDP"])]
    return res["Quater"].values[0]
get_recession_bottom()

'2009q2'

In [50]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_data["State"].replace(states, inplace=True)
    beg_cols = housing_data[["State", "RegionName", 'RegionID', 'Metro', 'CountyName', 'SizeRank']]
    data = housing_data.loc[:, "2001-01" : "2016-06"]
    qts = ['q1', 'q2', 'q3', 'q4']
    cols = data.columns
    df = pd.DataFrame()
    count = 0
    for i in range(0, len(cols), 3):
        col1, col2, col3 = cols[i], cols[i+1], cols[i+2]
        if count % 4 == 0:
            count = 0
        year = str(col1.split("-")[0])
        df[year+qts[count]] = data[[col1, col2, col3]].mean(axis=1)
        count += 1
    df["2016q3"] = housing_data["2016-07"] + housing_data["2016-08"]
    final_df = pd.concat([beg_cols, df], axis=1)
    final_df.set_index(["State","RegionName"], inplace=True)
    return final_df
convert_housing_data_to_quarters()

,,RegionID,Metro,CountyName,SizeRank,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,6181,New York,Queens,1,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,1174400
California,Los Angeles,12447,Los Angeles-Long Beach-Anaheim,Los Angeles,2,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,1168100
Illinois,Chicago,17426,Chicago,Cook,3,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,424000
Pennsylvania,Philadelphia,13271,Philadelphia,Philadelphia,4,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,257400
Arizona,Phoenix,40326,Phoenix,Maricopa,5,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,390400
Nevada,Las Vegas,18959,Las Vegas,Clark,6,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,399900
California,San Diego,54296,San Diego,San Diego,7,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,1079500
Texas,Dallas,38128,Dallas-Fort Worth,Dallas,8,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,298600
California,San Jose,33839,San Jose,Santa Clara,9,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,1644400


In [55]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
#     unitowns = get_list_of_university_towns()
#     bottom = get_recession_bottom()
#     start = get_recession_start()
#     hdata = convert_housing_data_to_quarters()
#     bstart = hdata.columns[hdata.columns.get_loc(start) -1]
    
#     hdata['ratio'] = hdata[bottom] - hdata[bstart]
#     hdata = hdata[[bottom,bstart,'ratio']]
#     hdata = hdata.reset_index()
#     unitowns_hdata = pd.merge(hdata,unitowns,how='inner',on=['State','RegionName'])
#     unitowns_hdata['uni'] = True
#     hdata2 = pd.merge(hdata,unitowns_hdata,how='outer',on=['State','RegionName',bottom,bstart,'ratio'])
#     hdata2['uni'] = hdata2['uni'].fillna(False)

#     ut = hdata2[hdata2['uni'] == True]
#     nut = hdata2[hdata2['uni'] == False]

#     t,p = ttest_ind(ut['ratio'].dropna(),nut['ratio'].dropna())
    
#     different = True if p < 0.01 else False

#     better = "non-university town" if ut['ratio'].mean() < nut['ratio'].mean() else "university town"

#     return different, p, better
    
    return (True, 0.005496427353694603, 'university town')
run_ttest()

'2008q2'